# Pixel Access and Cropping

Recall that pixels represent an ordered collection of measurements from a particular small region of Earth.
Each band is a particular measurement and all pixels, for a particular band, have the same _data_ being measured by that band.
For example, in a color image of Earth, the first band typically measures the amount of light in the _red wavelength_ range.
That is the Red band, followed by Green and Blue.

Based on the ground sample distance (GSD) of pixels, any particular sub-raster (AKA tile, chip, sub-image) from a larger image covers an area of Earth that is a subset of the Earth from the larger raster.
In this lab, we will access individual pixels and sub-rasters, as well as compute their coverage.

In [ ]:
###############-----------------------------
### Loading and Basic Visualization
############## -----------------------------

import rasterio
import numpy as np
%matplotlib inline
from rasterio.plot import show

# Open, and visualize with a little Band and Datatype information
with rasterio.open('/dsa/data/all_datasets/rasters/NED10Meter.tif') as raster_data:
    print("Number of bands: {}".format(raster_data.count))
    print("Data Types: {}".format(raster_data.dtypes))
    
    band1 = raster_data.read(1)
    rasterio.plot.show(band1, cmap='Greys')

## Examine the Spatial Characteristics

In [ ]:
print(raster_data.crs)
print(" ------------")
print(raster_data.crs.wkt)

In [ ]:
raster_data.bounds

We see that this image is in "UTM zone 13N" and that the Top-Left position is expected to be:   
`(X=467810.81 , Y=4399871.96)`

In the CRS, we see the unit of measure is: `UNIT["metre",1,AUTHORITY["EPSG","9001"]]` ... meaning meters!

So, the Upper-Left is over 467 KM away (to the right) from the centeral meridian (center vertical line) of the projection. 
It is also nearly 440 KM north of the projection's equator (center horizontal line).

---

## Affine Transform
The Affine Transform maps a raster's pixels from pixel cooridinate space (X,Y) with Y growing larger as you move down the raster to CRS coordinates (typically Y growing larger as you move North).
 * (More Mathy) - http://mathworld.wolfram.com/AffineTransformation.html
 * (Slightly Less Mathy) - https://en.wikipedia.org/wiki/Affine_transformation


In [ ]:
print(raster_data.affine)

The affine transform matrix allow us to use simple Python math operations to determine the position of a pixel on Earth that is selected from the raster.


You will notice in the affine matrix above that the Upper-Left is represented as the first two rows of the right-most column.
Therefore, if we multiply the raster size by the affine, e.g., `affine * size`, we should get the lower-right coordinates.

In [ ]:
raster_dims_xy = (raster_data.width, raster_data.height)

print("Compare Raster Dims from Above: {}".format(raster_dims_xy))
print("---------------------------------------------------------")
print("To Internal Raster Shape: {}".format(raster_data.shape))


#### Note: In square rasters this is not a problem, however under the hood the RasterIO library is using NumPy ND-Arrays and so major (first) index is Y or Row, not X or Column. 

In [ ]:
lower_right = raster_data.affine * raster_dims_xy
print("Lower-Right: {}".format(lower_right))

print("----------- COMPARE TO BOUNDS ------------------------")
print(raster_data.bounds)

So, if I want to know the position of the center (approximate)

In [ ]:
raster_center_xy = (raster_data.width/2, raster_data.height/2)
center = raster_data.affine * raster_center_xy
print("Center: {}".format(center))

## Reading values from the Raster

Any arbitrary pixel or window of pixels (sub-raster) can be accessed from the raster.
This can be done in two ways:
 1. Loading all the data, accessing the band, then using NumPy indexing to access values.
 1. Using the RasterIO `window` options to pull out sub-rasters
 
### NumPy pixel value access

In [ ]:
# NumPy Sub-array access
numpy_window = band1[256:768,256:768]


rasterio.plot.show(numpy_window, cmap='Greys')

In [ ]:
pixel_c = band1[512,512]
print("Center-ish pixel value = {}".format(pixel_c))

pixel_other = band1[200,512]
print("Other pixel value = {}".format(pixel_other))


### Sub-windows

A window is defined as follows:  
`((row_start, row_stop), (col_start, col_stop))`

The window is then an argument to the RasterIO dataset's band reading function.

In [ ]:
# Center Cropping, Y
y_size = raster_data.height
y_chip_base = raster_data.height/4 # go 1/4 down
y_chip_size = raster_data.height/2 # get sub-raster half height of original

# Center Cropping, X
x_size = raster_data.width
x_chip_base = raster_data.width/4 # go 1/4 to the right
x_chip_size = raster_data.width/2 # get sub-raster half width of original

# Buid our Window from base of window plus window size
y_range = (y_chip_base, y_chip_base + y_chip_size)  # defining a tuple
x_range = (x_chip_base, x_chip_base + x_chip_size)  # definign a tuple

subraster_window = (y_range, x_range)
print(subraster_window)

In [ ]:
with rasterio.open('/dsa/data/all_datasets/rasters/NED10Meter.tif') as raster_data:
    window = raster_data.read(1, window=subraster_window )
    print(window.shape)
    rasterio.plot.show(window, cmap='Greys')

### Ponder 

Geospatial raster files can exceed a billion pixels, e.g., a 1000 megapixel image.
Often, this may be memory prohibitive on a computing system you are using.
To get around the resource limits, you can use the Window-based access to rasters to read and then analyze/process/write a series of subrasters using Python loop constructs.

# Save your Notebook
## Then, Close and Halt